In [2]:
#importing necessary libraries
import numpy as np
import pandas as pd 
import seaborn as sns

In [3]:
# load the dataset 
data = pd.read_csv('Data_for_UCI_named.csv')

In [4]:
# drop 'stab' column as instructed
data = data.drop(columns=['stab'])
data.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,unstable


In [5]:
# assign features and the target class
X = data.drop(columns=['stabf'])
y = data['stabf']

#split the data into training and testing sets
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [6]:
#importing standard scaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

#use standard scaler to transform train and test
x_train_scaler = scaler.fit_transform(x_train)
x_test_scaler = scaler.transform(x_test)

**Question 1**

In [7]:
#calculate f1_score
tp = 355
tn = 120
fp = 1480
fn = 45
precision = tp / (tp + fp)
recall= tp / (tp + fn)
f1_score=  2*((precision* recall)/(precision+recall))

round(f1_score,4)

0.3177

**Question 3**

In [8]:
# calculate the recall
tp1 = 82
tn1 = 9
fp1 = 2
fn1 = 18
precision = tp1 / (tp1 + fp1)
recall= tp1 / (tp1 + fn1)
f1_score=  2*((precision* recall)/(precision+recall))
print('recall: %f' % recall)

recall: 0.820000


**Question 5**

In [9]:
#importing Random Forest classifier
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier(random_state=1)

#fit train set
RFC.fit(x_train_scaler, y_train)

# predict test set
RFC_pred = RFC.predict(x_test_scaler)

# accuracy of test set
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, confusion_matrix
RFC_accuracy = accuracy_score(y_true=y_test, y_pred=RFC_pred)
print('Accuracy: {}'.format(RFC_accuracy))

Accuracy: 0.929


**Question 12**

In [10]:
#importing LGBM classifier
from lightgbm import LGBMClassifier
LGBM = LGBMClassifier(random_state=1)

#fit train set
LGBM.fit(x_train_scaler, y_train)

# predict test set
LGBM_pred = LGBM.predict(x_test_scaler)

# accuracy of test set
LGBM_accuracy = accuracy_score(y_true=y_test, y_pred=LGBM_pred)
print('Accuracy: {}'.format(LGBM_accuracy))

Accuracy: 0.9375


**Question 17**

In [11]:
#importing XGBoost classifier
from xgboost import XGBClassifier
XGB = XGBClassifier(random_state=1)

#fit train set
XGB.fit(x_train_scaler, y_train)

# predict test set
XGB_pred = XGB.predict(x_test_scaler)

# accuracy of test set
XGB_accuracy = accuracy_score(y_true=y_test, y_pred=XGB_pred)
print('Accuracy: {}'.format(XGB_accuracy))

Accuracy: 0.9195


**Question 18**

In [12]:
# importing extra tree classifier
from sklearn.ensemble import ExtraTreesClassifier
ETC = ExtraTreesClassifier(random_state=1)

#fit train set
ETC.fit(x_train_scaler, y_train)

# predict test set
ETC_pred = ETC.predict(x_test_scaler)

# accuracy of test set
ETC_accuracy = accuracy_score(y_true=y_test, y_pred=ETC_pred)
print('Accuracy: {}'.format(ETC_accuracy))


Accuracy: 0.928


In [13]:
# range of parameters to find the best hyperparameters
n_estimators = [50, 100, 300, 500, 1000]
min_samples_split = [2, 3, 5, 7, 9]
min_samples_leaf = [1, 2, 4, 6, 8]
max_features = ['auto', 'sqrt', 'log2', None]

hyperparameter_grid = {'n_estimators': n_estimators,
                       'min_samples_leaf': min_samples_leaf,
                       'min_samples_split': min_samples_split,
                       'max_features': max_features}

#importing Random Search CV
from sklearn.model_selection import RandomizedSearchCV
clf =  RandomizedSearchCV(ETC,hyperparameter_grid, cv=5, n_iter=10, scoring = 'accuracy', n_jobs = -1, verbose = 1, random_state = 1)

#fit train set
searchCV = clf.fit(x_train_scaler, y_train)


Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  1.5min finished


In [14]:
# best hyperparameters from the randomized search CV
searchCV.best_params_

{'max_features': None,
 'min_samples_leaf': 8,
 'min_samples_split': 2,
 'n_estimators': 1000}

**Question 19**

In [15]:
#new optimal ExtraTreesClassifier
ETC1 = ExtraTreesClassifier(max_features=None,min_samples_leaf=8,min_samples_split=2,n_estimators=1000,random_state=1)

#fit train set
ETC1.fit(x_train_scaler, y_train)

# predict test set
ETC1_pred = ETC1.predict(x_test_scaler)

# accuracy of test set
ETC1_accuracy = accuracy_score(y_true=y_test, y_pred=ETC1_pred)
print('Accuracy: {}'.format(ETC1_accuracy))

Accuracy: 0.927


 The accuracy of the new optimal model is **lower** than the initial ExtraTreesClassifier model with no hyperparameter tuning

**Question 20**

In [16]:
feature = X.columns

# features importance
feat_importance = pd.DataFrame(ETC.feature_importances_,index=feature)
feat = feat_importance.sort_values(0)
feat


,0
p1,0.039507
p2,0.040371
p4,0.040579
p3,0.040706
g1,0.089783
g2,0.093676
g4,0.094019
g3,0.096883
tau3,0.113169
tau4,0.115466


In [17]:
# most important feature
print('most important feature: {}'.format(feat.idxmax()))

# least important feature
print('least important feature: {}'.format(feat.idxmin()))


most important feature: 0    tau2
dtype: object
least important feature: 0    p1
dtype: object
